In [1]:
import torch
from src.quadtree import Node
from src.utils.data import loadData
data_, _ = loadData(100)
node = Node(data_)

learning_rate = 1e-2
teacher_args_ = {
    "optimizer_lr": learning_rate,
    "epochs": 200,
    "times": 10,
    "train_data": torch.from_numpy(data_).float().to(),
    "alpha": 5,
    "beta": 10
}
sensitivity = 0.15
un_args_ = {
    "N": 100,
    "M": 10000 - 1,
    "epsilon": sensitivity,
    "x_area": [0, 300],
    "y_area": [0, 300],
    "model": None  # Unused param.
}

student_args_ = {
    "optimizer_lr": 1e-3,
    "epochs": 30000,
}

node.create_student(teacher_args_, un_args_, student_args_)

Loading data...
Data loaded.
Training Teacher Model
Epoch: 20/200..  Training loss: 8748.71191..  Reg Proj: 0.00000..  Reg Latent: 0.28135..  Memory: 8745.89844..  Cost: 8748.71191.. 
Epoch: 40/200..  Training loss: 11368.81543..  Reg Proj: 0.00000..  Reg Latent: 0.75179..  Memory: 11361.29785..  Cost: 11368.81543.. 
Epoch: 60/200..  Training loss: 8175.43115..  Reg Proj: 0.00000..  Reg Latent: 0.08147..  Memory: 8174.61621..  Cost: 8175.43115.. 
Epoch: 80/200..  Training loss: 7308.98291..  Reg Proj: 0.00000..  Reg Latent: 0.08837..  Memory: 7308.09912..  Cost: 7308.98291.. 
Epoch: 100/200..  Training loss: 7540.31885..  Reg Proj: 0.00000..  Reg Latent: 0.07802..  Memory: 7539.53857..  Cost: 7540.31885.. 
Epoch: 120/200..  Training loss: 8342.84473..  Reg Proj: 0.00000..  Reg Latent: 0.11079..  Memory: 8341.73633..  Cost: 8342.84473.. 
Epoch: 140/200..  Training loss: 7948.96338..  Reg Proj: 0.00000..  Reg Latent: 0.16805..  Memory: 7947.28271..  Cost: 7948.96338.. 
Epoch: 160/200..  

/home/getekid/Sites/athenarc/GeoCluster/src/utils/functions.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  E = Linf_array(torch.tensor(n_points), torch.tensor(outputs))


Processing...
flag is 1
m is 10000
i is 10000


/home/getekid/Sites/athenarc/GeoCluster/src/utils/functions.py:117: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  qp = torch.tensor(qp)
/home/getekid/Sites/athenarc/GeoCluster/src/utils/functions.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = torch.tensor(outputs)


Loading data...
Data loaded.


/home/getekid/Sites/athenarc/GeoCluster/src/quadtree.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  qp=torch.tensor(qp),
/home/getekid/Sites/athenarc/GeoCluster/src/models.py:272: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  qp = torch.tensor(qp).to(device)
/home/getekid/Sites/athenarc/GeoCluster/src/models.py:281: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  qp = torch.tensor(qp, dtype=torch.float32).to(device)


Training Student Model
Acc:  0.3321
Epoch:  0 Cost:  1395.1690673828125


KeyboardInterrupt: 